In [1]:
from google.colab import drive
drive.mount("/content/gdrive",force_remount=True)

Mounted at /content/gdrive


In [2]:
zip_path = '/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/taOUHands.zip'
!unzip -q '/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/taOUHands.zip' 

In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
import cv2
import numpy as np
import os
from tensorflow.keras.applications import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import random
import albumentations as A

In [12]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    zoom_range = [0.7, 1.7],
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = [0.5, 1.5]
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_data = train_datagen.flow_from_directory("/content/taOUHands/train/re1", target_size = (256, 256), batch_size = 32, class_mode ="categorical")
val_data = test_datagen.flow_from_directory("/content/taOUHands/val/re1", target_size = (256, 256), batch_size = 32, class_mode ="categorical")
test_data = test_datagen.flow_from_directory("/content/taOUHands/test/re1", target_size = (256, 256), batch_size = 32, class_mode ="categorical")

Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [18]:
def build_MobileNetV2(img_size = 256):
  inputs = Input(shape=(img_size, img_size, 3))
  base_model = MobileNetV2(input_tensor= inputs, weights= "imagenet", include_top= False, alpha= 0.5)
  x = base_model.get_layer('block_10_project_BN').output
  x = Conv2D(32, 1, padding='same', kernel_initializer='he_normal')(x)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = GlobalAveragePooling2D()(x)
  output = Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
  model = Model(inputs=base_model.input, outputs=output)
  model.summary()
  return model

In [19]:
model = build_MobileNetV2()

3211264/3201480 [==============================] - 0s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 16  432         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 16  64          ['Conv1[0][0]']                  
                             

In [41]:
model.compile(loss = "categorical_crossentropy", optimizer = Adam(0.001), metrics = ["accuracy"])
callbacks = [keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/code/BestResultClass/OuHands_{epoch}_{val_loss: .5f}_{val_accuracy: .4f}.hdf5", monitor = "val_accuracy", save_best_only = True, mode = "max"), ReduceLROnPlateau(monitor='val_loss', patience=20, verbose=1, factor=0.1, min_lr=0.000001)]
epochs = 500
model.fit(train_data, validation_data = val_data, callbacks = callbacks, verbose = 1, epochs = epochs)

Epoch 1/500
50/50 [==============================] - 34s 614ms/step - loss: 0.0181 - accuracy: 0.9994 - val_loss: 0.2645 - val_accuracy: 0.9250 - lr: 1.0000e-05
Epoch 2/500
50/50 [==============================] - 30s 591ms/step - loss: 0.0179 - accuracy: 0.9987 - val_loss: 0.2625 - val_accuracy: 0.9250 - lr: 1.0000e-05
Epoch 3/500
50/50 [==============================] - 29s 588ms/step - loss: 0.0239 - accuracy: 0.9987 - val_loss: 0.2620 - val_accuracy: 0.9250 - lr: 1.0000e-05
Epoch 4/500
50/50 [==============================] - 29s 587ms/step - loss: 0.0182 - accuracy: 0.9994 - val_loss: 0.2664 - val_accuracy: 0.9225 - lr: 1.0000e-05
Epoch 5/500
34/50 [===================>..........] - ETA: 9s - loss: 0.0157 - accuracy: 1.0000

KeyboardInterrupt: ignored

In [40]:
model = load_model("/content/gdrive/MyDrive/Teaching Assistant /week 8/HandGesture/code/BestResultClass/OuHands_8_ 0.26329_ 0.9300_8000.hdf5")

In [39]:
model.evaluate(test_data)

32/32 [==============================] - 3s 82ms/step - loss: 0.6992 - accuracy: 0.8000


[0.6991776823997498, 0.800000011920929]